Importo le principali librerie e inizializzo il dataframe su cui devo lavorare

In [1]:
import pandas as pd 
import numpy as np

planets = pd.read_csv('kepler_confirmed_planets.csv', sep=',',  index_col=0)
planets

,pl_hostname,pl_letter,pl_discmethod,pl_pnum,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,...,st_masserr1,st_masserr2,st_masslim,st_massblend,st_rad,st_raderr1,st_raderr2,st_radlim,st_radblend,rowupdate
rowid,,,,,,,,,,,,,,,,,,,,,
1,11 Com,b,Radial Velocity,1,326.030000,0.320000,-0.320000,0.0,1.290000,0.050000,...,0.30,-0.30,0.0,0.0,19.00,2.00,-2.00,0.0,0.0,2014-05-14
2,11 UMi,b,Radial Velocity,1,516.220000,3.250000,-3.250000,0.0,1.540000,0.070000,...,0.25,-0.25,0.0,0.0,24.08,1.84,-1.84,0.0,0.0,2014-05-14
3,14 And,b,Radial Velocity,1,185.840000,0.230000,-0.230000,0.0,0.830000,NaN,...,0.10,-0.20,0.0,0.0,11.00,1.00,-1.00,0.0,0.0,2014-05-14
4,14 Her,b,Radial Velocity,1,1773.400000,2.500000,-2.500000,0.0,2.770000,0.050000,...,0.05,-0.05,0.0,0.0,NaN,NaN,NaN,NaN,0.0,2014-05-14
5,16 Cyg B,b,Radial Velocity,1,798.500000,1.000000,-1.000000,0.0,1.681000,0.097000,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,2015-09-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3368,ups And,b,Radial Velocity,4,4.617033,0.000023,-0.000023,0.0,0.059222,0.000000,...,NaN,NaN,0.0,0.0,1.56,NaN,NaN,0.0,0.0,2014-05-14
3369,ups And,c,Radial Velocity,4,241.258000,0.064000,-0.064000,0.0,0.827774,0.000015,...,NaN,NaN,0.0,0.0,1.56,NaN,NaN,0.0,0.0,2014-05-14
3370,ups And,d,Radial Velocity,4,1276.460000,0.570000,-0.570000,0.0,2.513290,0.000750,...,NaN,NaN,0.0,0.0,1.56,NaN,NaN,0.0,0.0,2014-05-14


calcolo della luminosità della stella (per calcolo della zona abitabile):
La luminosità di una stella può essere calcolata utilizzando la legge di Stefan-Boltzmann, che stabilisce che la luminosità (L) è proporzionale alla quarta potenza della sua temperatura superficiale (T). La formula per la luminosità stellare è la seguente:

$$ L = 4\pi R^2 \sigma T^4 $$

Dove:
L è la luminosità della stella (in watt),
R è il raggio della stella (in metri),
σ è la costante di Stefan-Boltzmann (5.67 × 10^-8 watt per metro quadrato per Kelvin alla quarta potenza),
T è la temperatura superficiale della stella (in Kelvin).
In questa formula, si suppone che la stella sia una sfera perfetta. Questa è una formula generale per calcolare la luminosità di una stella in base al suo raggio e alla sua temperatura superficiale.


In [4]:

sigma = 5.67e-8

planets['st_bright'] = 4 * np.pi * (planets['st_rad']**2) * sigma * (planets['st_teff']**4)
planets['st_bright']

rowid
1       1.300608e+11
2       1.465767e+11
3       4.626385e+10
4                NaN
5                NaN
            ...     
3368             NaN
3369             NaN
3370             NaN
3371             NaN
3372    5.356326e+10
Name: st_bright, Length: 3372, dtype: float64

In [5]:
sun_bright = 3.828e26

# internal and external radius of the habitable zone
planets['R_int'] = 0.72 * (np.sqrt(planets['st_bright']) / sun_bright)  
planets['R_est'] = 1.77 * (np.sqrt(planets['st_bright']) / sun_bright)  

dopo aver calcolato la luminosità di ogni stella e il raggio interno ed esterno della sua zona abitabile passo a controllare se ogni pianeta che orbita attorno ad una stella data si trova all'interno di essa